This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# 自然語言處理基礎功（二）（參考答案）

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
`工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`

## 練習題：誰是人氣王

如果是一個關心棒球發展，但是沒有太多時間看比賽的人，對於一直新出現的球員名字可能難以記住，而且也很難確定目前媒體的當紅球星到底有誰。要解決這個問題，一個方式就是自動去新聞找可能的球員名字，並且統計出現的次數。在這個練習，我們要用中華職棒的新聞標題，去挖掘哪個球員是當紅的人氣王！

新聞來源為 [ETtoday 運動雲](https://sports.ettoday.net/news-list/%E6%A3%92%E7%90%83/%E4%B8%AD%E8%81%B7/1)，資料的時間範圍為 2021.06.16 到 2021.06.26，總共有 120 篇。

![](https://i.imgur.com/qQUoabT.png)

（[圖片來源](https://www.tsna.com.tw/tw/news/show.php?type=1&num=39608)）

In [ ]:
!pip install ckiptagger[tf,gdown]

In [ ]:
from ckiptagger import POS
from ckiptagger import NER

### Step 1. 命名實體辨識

為了要做繁體中文的命名實體辨識（NER），我們使用中研院所研發的 [CkipTagger](https://github.com/ckiplab/ckiptagger)。

首先，我們要先下載這個套件所需要的相關模型檔案：`data.zip` 到 Colab Server 的硬碟，並且解壓縮後，會看到 `data` 這個資料夾，底下有相關的模型檔案。

In [ ]:
!wget http://ckip.iis.sinica.edu.tw/data/ckiptagger/data.zip -O data.zip && unzip -qo data.zip

In [ ]:
!ls data

接下來我們讀取 NER 的模型，由於做 NER 之前必須要有詞性標記（POS Tagging）的資訊，因此同時也讀取 POS Tagging 的模型。

In [ ]:
pos = POS("./data")
ner = NER("./data")

In [ ]:
sentences = ["大谷翔平再現超狂跑速！ 奔出4.04秒內野安打"]

In [ ]:
pos_sentences = pos(sentences)
entity_sentences = ner(sentences, pos_sentences)

In [ ]:
for entity in entity_sentences[0]:
    print(entity)

## Step 2. 找出人氣王

我們先下載中華職棒新聞標題的檔案（`cpbl.txt`），並且讀取它。這個檔案每行就是一個新聞標題，因此我們必須把每行分開，並且把每行的內容存到一個 list （`sentences`）裡。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/baseball_news/npb_kbo.txt -O npb_kbo.txt

In [ ]:
with open("npb_kbo.txt", "r") as f:
    full_text = f.read()
    print(full_text)

In [ ]:
sentences = full_text.split("\n")
print(sentences)

接下來，我們定義 function `parse_people()`，這個 function 會用 NER 模型把每個新聞標題的人名給找出來，並且去計算說每個人名出現過幾次。

In [ ]:
def parse_people(sentences):
    pos_sentences = pos(sentences)
    entity_sentences = ner(sentences, pos_sentences)

    person_count = {}
    for entity_sentence in entity_sentences:
        for entity in entity_sentence:
            entity_type = entity[2]
            entity_content = entity[3]
            if entity_type == "PERSON" and len(entity_content) > 1:
                if entity_content not in person_count:
                    person_count[entity_content] = 1
                else:
                    person_count[entity_content] += 1

    people = [(name, count) for name, count in person_count.items()]

    return people

In [ ]:
people = parse_people(sentences)
print(people)

再來就是要秀出出現過最多次的前 10 個人名。這裡我們定義 function `show_top()` 來做這件事情，作法就是先做排序（sorting），再把前 10 個人名以及出現的次數秀出來，就大功告成！

In [ ]:
def show_top(people, topk=10):
    people_sorted = sorted(people, key=lambda p: p[1], reverse=True)
    for person in people_sorted[:topk]:
        print("{}: {}".format(person[0], person[1]))

In [ ]:
show_top(people, topk=15)

練習時間：
1. 請將原本中華職棒的新聞資料，改成用日職、韓職的新聞資料（[連結](https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/baseball_news/npb_kbo.txt)），重新跑一次結果看看。
2. 上面的程式碼，有些會出現不合理的人名，例如 `林`。我們定義當名字只有一個字的時候，為不合理的人名，請試著修改 `parse_people()` 這個 function，將不合理的人名去除。
3. 如果我們現在想要改成：秀出出現次數前 15 名的人名，請問要如何修改？
4. 請試著修改下面的 function `show_bottom`，這個 function 可以秀出 10 個名字最少出現的名字。

In [ ]:
def show_bottom(people, bottomk=10):
    # PLEASE MODIFY CODE BELOW:
    people_sorted = sorted(people, key=lambda p: p[1])
    for person in people_sorted[:bottomk]:
        print("{}: {}".format(person[0], person[1]))

In [ ]:
show_bottom(people)